In [2]:
!pip install pgmpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [9]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [10]:
# Define the Bayesian Network structure
model = BayesianNetwork([('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'D')])

In [11]:
# Define Conditional Probability Distributions (CPDs) for each variable
# Variable A (Patient Condition)
cpd_patient_condition = TabularCPD(variable='A', variable_card=3,
                                    values=[[0.7], [0.2], [0.1]],
                                    state_names={'A': ['good', 'fair', 'poor']})


In [12]:
# Variable B (Test Result)
cpd_test_result = TabularCPD(variable='B', variable_card=2,
                              values=[[0.8, 0.5, 0.3],
                                      [0.2, 0.5, 0.7]],
                              evidence=['A'],
                              evidence_card=[3],
                              state_names={'B': ['positive', 'negative'],
                                           'A': ['good', 'fair', 'poor']})


In [13]:
# Variable C (Treatment Decision)
cpd_treatment_decision = TabularCPD(variable='C', variable_card=2,
                                    values=[[0.9, 0.7, 0.5, 0.3, 0.1, 0.1],
                                            [0.1, 0.3, 0.5, 0.7, 0.9, 0.9]],
                                    evidence=['A', 'B'],
                                    evidence_card=[3, 2],
                                    state_names={'C': ['treated', 'not treated'],
                                                 'A': ['good', 'fair', 'poor'],
                                                 'B': ['positive', 'negative']})


In [14]:
# Variable D (Outcome)
cpd_outcome = TabularCPD(variable='D', variable_card=2,
                         values=[[0.8, 0.2],
                                 [0.2, 0.8]],
                         evidence=['C'],
                         evidence_card=[2],
                         state_names={'D': ['positive', 'negative'],
                                      'C': ['treated', 'not treated']})

In [15]:
# Add CPDs to the model
model.add_cpds(cpd_patient_condition, cpd_test_result, cpd_treatment_decision, cpd_outcome)


In [16]:
# Check if the model is correct
model.check_model()

True

In [17]:
# Perform inference using Variable Elimination
inference = VariableElimination(model)

In [19]:
# Query 1: Probability of patient condition A
query_patient_condition = inference.query(variables=['A'])


In [20]:
# Query 2: Probability distribution of the test result B given patient's condition A is poor
query_test_result_given_poor_condition = inference.query(variables=['B'], evidence={'A': 'poor'})

In [21]:
# Query 3: Probability distribution of the treatment decision C given patient's condition A is fair and test result B is negative
query_treatment_decision_given_fair_condition_and_negative_test = inference.query(variables=['C'],
                                                                                  evidence={'A': 'fair', 'B': 'negative'})


In [22]:
# Query 4: Probability distribution of the outcome D given patient's condition A is fair, test result B is positive, and treatment decision C is not treated
query_outcome_given_fair_condition_positive_test_not_treated = inference.query(variables=['D'],
                                                                               evidence={'A': 'fair', 'B': 'positive', 'C': 'not treated'})

In [23]:
# Query 5: Most likely outcome D given observed values of variables A, B, and C
query_most_likely_outcome_given_observed_values = inference.map_query(variables=['D'],
                                                                      evidence={'A': 'good', 'B': 'positive', 'C': 'treated'})

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [24]:
# Query 6: Predicting treatment decision C given patient's condition and the test result
query_predicted_treatment_decision = inference.map_query(variables=['C'], evidence={'A': 'good', 'B': 'positive'})

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [25]:
# Query 7: Sensitivity analysis of outcome D to changes in treatment decision C
sensitivity_analysis = []
for treatment in ['treated', 'not treated']:
    query_outcome_given_treatment = inference.map_query(variables=['D'], evidence={'C': treatment})
    sensitivity_analysis.append((treatment, query_outcome_given_treatment))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [26]:
# Query 8: Probability distribution of the patient's outcome D under varying conditions for variables A, B, and C
outcome_under_varying_conditions = []
for condition in ['good', 'fair', 'poor']:
    for test_result in ['positive', 'negative']:
        for treatment_decision in ['treated', 'not treated']:
            query_outcome_given_conditions = inference.map_query(variables=['D'],
                                                                  evidence={'A': condition, 'B': test_result, 'C': treatment_decision})
            outcome_under_varying_conditions.append((condition, test_result, treatment_decision, query_outcome_given_conditions))


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [27]:
# Print query results
print("Query 1 Result:\n", query_patient_condition)
print("\nQuery 2 Result:\n", query_test_result_given_poor_condition)
print("\nQuery 3 Result:\n", query_treatment_decision_given_fair_condition_and_negative_test)
print("\nQuery 4 Result:\n", query_outcome_given_fair_condition_positive_test_not_treated)
print("\nQuery 5 Result:\n", query_most_likely_outcome_given_observed_values)
print("\nQuery 6 Result:\n", query_predicted_treatment_decision)
print("\nQuery 7 Result (for treated):\n")
for treatment, outcome in sensitivity_analysis:
    print(f"Treatment: {treatment}, Outcome: {outcome}")
print("\nQuery 8 Result under varying conditions: ")
for condition, test_result, treatment, outcome in outcome_under_varying_conditions:
    print(f"Condition: {condition}, Test Result: {test_result}, Treatment: {treatment}, Outcome: {outcome}")

Query 1 Result:
 +---------+----------+
| A       |   phi(A) |
+=========+==========+
| A(good) |   0.7000 |
+---------+----------+
| A(fair) |   0.2000 |
+---------+----------+
| A(poor) |   0.1000 |
+---------+----------+

Query 2 Result:
 +-------------+----------+
| B           |   phi(B) |
+=============+==========+
| B(positive) |   0.3000 |
+-------------+----------+
| B(negative) |   0.7000 |
+-------------+----------+

Query 3 Result:
 +----------------+----------+
| C              |   phi(C) |
+================+==========+
| C(treated)     |   0.3000 |
+----------------+----------+
| C(not treated) |   0.7000 |
+----------------+----------+

Query 4 Result:
 +-------------+----------+
| D           |   phi(D) |
+=============+==========+
| D(positive) |   0.2000 |
+-------------+----------+
| D(negative) |   0.8000 |
+-------------+----------+

Query 5 Result:
 {'D': 'positive'}

Query 6 Result:
 {'C': 'treated'}

Query 7 Result (for treated):

Treatment: treated, Outcome: {'